In [2]:
import openai
import pandas as pd
import chromadb
from chromadb.config import Settings
import streamlit as st
import os
from dotenv import load_dotenv
from pathlib import Path
# Set your OpenAI API key


In [3]:
import pandas as pd
from datasets import load_dataset

import pandas as pd

# Example local path: "train-00000-of-00001.parquet"
# If it's in the same directory, you can do:
df = pd.read_parquet("train-00000-of-00001.parquet")

df.head()










c:\Users\nakul\langchain_2\myvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,context,question,answers
0,The patient was admitted on 5/5/2006 with a hi...,What is her current dose of lasix ( furosemide ),"{'answer_end': [2598], 'answer_start': [2564],..."
1,This 57-year-old female with a distant history...,Is the patient currently or have they ever tak...,"{'answer_end': [541], 'answer_start': [532], '..."
2,An 81-year-old woman with Atrial Fibrillation ...,Has a patient had morphine,"{'answer_end': [1342], 'answer_start': [1319],..."
3,A 59 year-old woman with metastatic breast can...,Why was tessalon perles ( benzonatate ) origin...,"{'answer_end': [1007], 'answer_start': [988], ..."
4,Ms. Elter is an 83-year-old Spanish-speaking f...,Is there history of use of allopurinol,"{'answer_end': [539], 'answer_start': [475], '..."


In [53]:
df.head()

,patient_id,notes
0,0,The patient was admitted on 5/5/2006 with a hi...
1,1,This 57-year-old female with a distant history...
2,2,An 81-year-old woman with Atrial Fibrillation ...
3,3,A 59 year-old woman with metastatic breast can...
4,4,Ms. Elter is an 83-year-old Spanish-speaking f...


In [62]:
df["notes"][0]

"The patient was admitted on 5/5/2006 with a history of mechanical fall, with the attending physician being Dr. Clemente Armand Bolstad, with a full code status and disposition of Rehabilitation. Medications on Admission included Amiodarone 100 QD, Colace 100 bid, lasix 40mg QD, Glyburide 5mg bid, Plaquenil 200mg bid, Isordil 20mg tid, Lisinopril 20mg QD, Coumadin 5mg 3dys/week, 2.5mg 4dys/week, Norvasc 10mg QD, Neurontin 300mg TID, with APAP prn. An override was added on 10/2/06 by Gerad E. Dancy, PA for POTENTIALLY SERIOUS INTERACTION: AMIODARONE HCL & WARFARIN with the reason for override being monitoring. The patient was rehydrated with IVF and PO's were encouraged, holding Glypizide while in house, Novolog sliding scale was started on 1/2, Low dose NPH 6 units BID was started on 1/2, bridged with lovenox and INR therapeutic 1/2 and restarted on home regimen of 5/2.5mg variable dose. Pain was controlled with TYLENOL (ACETAMINOPHEN) 650 MG PO Q4H PRN Pain, Headache. A CT pelvis show

In [57]:
import pandas as pd

# Allow unlimited column width
pd.set_option('display.max_colwidth', None)


In [61]:
from IPython.display import display, HTML

# Set CSS to wrap text within DataFrame cells
display(HTML("""
<style>
    .dataframe td {
        white-space: pre-wrap;
        word-wrap: break-word;
    }
</style>
"""))


# Set the display width (adjust as needed for your console or notebook)
pd.set_option('display.width', 80)

# If set to False, prevents pandas from expanding to the full console width
pd.set_option('display.expand_frame_repr', False)


In [64]:
df["notes"][0]

"The patient was admitted on 5/5/2006 with a history of mechanical fall, with the attending physician being Dr. Clemente Armand Bolstad, with a full code status and disposition of Rehabilitation. Medications on Admission included Amiodarone 100 QD, Colace 100 bid, lasix 40mg QD, Glyburide 5mg bid, Plaquenil 200mg bid, Isordil 20mg tid, Lisinopril 20mg QD, Coumadin 5mg 3dys/week, 2.5mg 4dys/week, Norvasc 10mg QD, Neurontin 300mg TID, with APAP prn. An override was added on 10/2/06 by Gerad E. Dancy, PA for POTENTIALLY SERIOUS INTERACTION: AMIODARONE HCL & WARFARIN with the reason for override being monitoring. The patient was rehydrated with IVF and PO's were encouraged, holding Glypizide while in house, Novolog sliding scale was started on 1/2, Low dose NPH 6 units BID was started on 1/2, bridged with lovenox and INR therapeutic 1/2 and restarted on home regimen of 5/2.5mg variable dose. Pain was controlled with TYLENOL (ACETAMINOPHEN) 650 MG PO Q4H PRN Pain, Headache. A CT pelvis show

In [65]:
text = df['notes'].iloc[0]
print("\n".join(text[i:i+80] for i in range(0, len(text), 80)))


The patient was admitted on 5/5/2006 with a history of mechanical fall, with the
 attending physician being Dr. Clemente Armand Bolstad, with a full code status 
and disposition of Rehabilitation. Medications on Admission included Amiodarone 
100 QD, Colace 100 bid, lasix 40mg QD, Glyburide 5mg bid, Plaquenil 200mg bid, I
sordil 20mg tid, Lisinopril 20mg QD, Coumadin 5mg 3dys/week, 2.5mg 4dys/week, No
rvasc 10mg QD, Neurontin 300mg TID, with APAP prn. An override was added on 10/2
/06 by Gerad E. Dancy, PA for POTENTIALLY SERIOUS INTERACTION: AMIODARONE HCL & 
WARFARIN with the reason for override being monitoring. The patient was rehydrat
ed with IVF and PO's were encouraged, holding Glypizide while in house, Novolog 
sliding scale was started on 1/2, Low dose NPH 6 units BID was started on 1/2, b
ridged with lovenox and INR therapeutic 1/2 and restarted on home regimen of 5/2
.5mg variable dose. Pain was controlled with TYLENOL (ACETAMINOPHEN) 650 MG PO Q
4H PRN Pain, Headache. A CT 

In [6]:
df_patient_notes = df["context"]

In [7]:
df_patient_notes.shape

(130956,)

In [8]:
df_patient_notes = df_patient_notes.iloc[:50]

In [52]:
df_patient_notes

0     The patient was admitted on 5/5/2006 with a hi...
1     This 57-year-old female with a distant history...
2     An 81-year-old woman with Atrial Fibrillation ...
3     A 59 year-old woman with metastatic breast can...
4     Ms. Elter is an 83-year-old Spanish-speaking f...
5     This 62-year-old white male with insulin depen...
6     Patient Alequin, Garland, a 57-year-old female...
7     This is a 70-year-old woman with ischemic card...
8     Justin Eans, a 56 year old patient with a hist...
9     GVERRERO , STAN O 346-21-49-8, a 74 yo woman i...
10    Stansbury Ellsworth, a 59-year-old female with...
11    This 90+-year-old male with a complex past med...
12    Glen Cooperwood, a 69 year old male with ische...
13    The patient is an 83-year-old female with a hi...
14    This 62-year-old white male with insulin depen...
15    GVERRERO , STAN O 346-21-49-8, a 74 yo woman i...
16    This 70-year-old female with CHF, coronary art...
17    The patient had continued to remain stable

In [49]:


# Set maximum number of rows displayed
pd.set_option('display.max_rows', 100)  # change 100 to your desired number

# Set maximum number of columns displayed
pd.set_option('display.max_columns', 50)  # change 50 to your desired number


In [50]:
df_patient_notes[0]

"The patient was admitted on 5/5/2006 with a history of mechanical fall, with the attending physician being Dr. Clemente Armand Bolstad, with a full code status and disposition of Rehabilitation. Medications on Admission included Amiodarone 100 QD, Colace 100 bid, lasix 40mg QD, Glyburide 5mg bid, Plaquenil 200mg bid, Isordil 20mg tid, Lisinopril 20mg QD, Coumadin 5mg 3dys/week, 2.5mg 4dys/week, Norvasc 10mg QD, Neurontin 300mg TID, with APAP prn. An override was added on 10/2/06 by Gerad E. Dancy, PA for POTENTIALLY SERIOUS INTERACTION: AMIODARONE HCL & WARFARIN with the reason for override being monitoring. The patient was rehydrated with IVF and PO's were encouraged, holding Glypizide while in house, Novolog sliding scale was started on 1/2, Low dose NPH 6 units BID was started on 1/2, bridged with lovenox and INR therapeutic 1/2 and restarted on home regimen of 5/2.5mg variable dose. Pain was controlled with TYLENOL (ACETAMINOPHEN) 650 MG PO Q4H PRN Pain, Headache. A CT pelvis show

In [10]:
df = df_patient_notes.reset_index()

In [11]:
df.rename(columns={"index": "patient_id", "context": "notes"}, inplace=True)

In [12]:
df.head()

,patient_id,notes
0,0,The patient was admitted on 5/5/2006 with a hi...
1,1,This 57-year-old female with a distant history...
2,2,An 81-year-old woman with Atrial Fibrillation ...
3,3,A 59 year-old woman with metastatic breast can...
4,4,Ms. Elter is an 83-year-old Spanish-speaking f...


In [13]:
df.shape

(50, 2)

## Indexing the documents with chromaDB vector database

In [14]:
import chromadb
import openai

# Initialize Chroma client
client = chromadb.Client()

# Check if the collection already exists
if "patient_notes" in [c.name for c in client.list_collections()]:
    collection = client.get_collection("patient_notes")  # Retrieve the existing collection
else:
    collection = client.create_collection(name="patient_notes")  # Create a new collection


# Prepare the notes column for embeddings
embeddings = []
ids = []
metadatas = []

for i, row in df.iterrows():
    # Generate embedding for the 'notes' column
    embedding = openai.Embedding.create(input=row['notes'], model="text-embedding-ada-002")['data'][0]['embedding']
    embeddings.append(embedding)
    ids.append(str(row['patient_id']))  # Assuming 'patient_id' column exists
    metadatas.append({"patient_id": row['patient_id']})

# Add the embeddings to ChromaDB
collection.add(
    embeddings=embeddings,
    documents=df['notes'].tolist(),  # Use 'notes' instead of 'context'
    ids=ids,
    metadatas=metadatas
)




In [29]:
metadatas[0]

{'patient_id': 0}

## Building RAG

In [25]:
# Retriever Function
def retrieve_notes(query: str, patient_id: str = None, top_k: int = 3):
    """
    Retrieve relevant notes from ChromaDB based on the query.
    If patient_id is provided, restrict the query to that patient's note.
    """
    if patient_id:
        # Retrieve the specific patient's note
        results = collection.get(
            where={"patient_id": patient_id},
            include=["documents", "metadatas"]
        )
        if results['documents']:
            return results['documents'][0]  # Return the single note
        else:
            return None  # No note found for this patient
    else:
        # General query across all patient notes
        # Generate embedding for the query
        query_embedding = openai.Embedding.create(
            input=query,
            model="text-embedding-ada-002"
        )['data'][0]['embedding']
        
        # Perform similarity search
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k,
            include=["documents", "metadatas"]
        )
        
        # Flatten the results
        retrieved_notes = []
        for doc, meta in zip(results['documents'][0], results['metadatas'][0]):
            retrieved_notes.append({
                "patient_id": meta['patient_id'],
                "note": doc
            })
        return retrieved_notes



In [19]:

# Prompt Generation Function
def generate_prompt(query: str, retrieved_notes):
    """
    Generate a prompt for the language model based on retrieved notes.
    """
    if isinstance(retrieved_notes, list):
        # General query with multiple notes
        context = "\n\n".join([f"Patient ID {item['patient_id']}: {item['note']}" for item in retrieved_notes])
        prompt = f"""
        You are a helpful medical assistant. The following are patient notes that may contain relevant information:

        {context}

        Based on the above notes, please answer the following question:
        "{query}"
        """
    else:
        # Patient-specific query with a single note
        context = retrieved_notes
        prompt = f"""
        You are a helpful medical assistant. The following is the patient's note:

        {context}

        Based on the above note, please answer the following question:
        "{query}"
        """
    return prompt

In [20]:
def get_answer(prompt: str):
    """
    Use OpenAI GPT to generate an answer based on the provided prompt.
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful medical assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=300
    )
    return response['choices'][0]['message']['content'].strip()

In [21]:
# Language Model Query Function
def get_answer(prompt: str):
    """
    Use OpenAI GPT to generate an answer based on the provided prompt.
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful medical assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
        max_tokens=300
    )
    return response['choices'][0]['message']['content'].strip()

In [22]:
# RAG Pipeline Function
def ask_question(query: str, patient_id: str = None, top_k: int = 3):
    """
    RAG pipeline:
    1. Retrieve relevant notes from ChromaDB.
    2. Generate a prompt with the retrieved notes.
    3. Generate an answer using the language model.
    
    Parameters:
    - query (str): The question to ask.
    - patient_id (str, optional): The patient ID to restrict the query. Defaults to None.
    - top_k (int, optional): Number of top results to retrieve for general queries. Defaults to 3.
    
    Returns:
    - str: The answer generated by the language model.
    """
    # Step 1: Retrieve notes
    retrieved = retrieve_notes(query, patient_id, top_k)
    
    if not retrieved:
        if patient_id:
            return f"No notes found for patient_id: {patient_id}"
        else:
            return "No relevant notes found for this query."
    
    # Step 2: Generate prompt
    prompt = generate_prompt(query, retrieved)
    
    # Step 3: Get answer
    answer = get_answer(prompt)
    
    return answer

In [46]:
# Example Usage
if __name__ == "__main__":
        # Example 1: Patient-Specific Query
    query1 = "First give me a short summary of the patient. Then what has been the history of medications and diseases of this patient? Give in bullet points, incldue age, gender etc."
    patient_id1 = 10 # Replace with an actual patient_id as a string
    response1 = ask_question(query=query1, patient_id=patient_id1)
    print(f"Response for Patient ID {patient_id1}:\n{response1}\n")




Response for Patient ID 10:
Summary of the Patient:
- Stansbury Ellsworth is a 59-year-old female who was admitted with atypical chest pain.
- She has a history of NIDDM (Non-Insulin Dependent Diabetes Mellitus), GERD (Gastroesophageal Reflux Disease), HTN (Hypertension), Depression, and known CAD (Coronary Artery Disease) s/p circumflex stent in 2002.

History of Medications and Diseases:
- Age: 59 years
- Gender: Female
- Diseases:
  - NIDDM (Non-Insulin Dependent Diabetes Mellitus)
  - GERD (Gastroesophageal Reflux Disease)
  - HTN (Hypertension)
  - Depression
  - Known CAD (Coronary Artery Disease) s/p circumflex stent in 2002
- Medications:
  - CLONAZEPAM 0.5 MG PO QD
  - LISINOPRIL 5 MG PO QD
  - POTASSIUM CHLORIDE IV
  - POTASSIUM CHLORIDE PO
  - MOM (MAGNESIUM HYDROXIDE) 30 MILLILITERS PO QD
  - MAALOX-TABLETS QUICK DISSOLVE/CHEWABLE 1-2 TAB PO Q6H
  - PROTONIX (PANTOPRAZ



In [39]:
if __name__ == "__main__":

    
    # Example 2: General Query Across All Patients
    query2 = "whats the most common type of cancer?"
    response2 = ask_question(query=query2, top_k=23)
    print(f"Response for General Query:\n{response2}\n")

Response for General Query:
Based on the patient notes provided, the most common type of cancer mentioned is breast cancer.

